In [1]:
%load_ext autoreload
%autoreload 2

In [6]:
from elasticsearch import Elasticsearch

es_client = Elasticsearch(['http://localhost:9200'])

In [5]:
import eland as ed

In [2]:
import pandas as pd

In [3]:
df = pd.DataFrame(
    {
        "process.name": ["process1"],
        "file.hash.sha256": ["hash1"],
        "file.hash.sha1": ["hash2"],
        "label": [1],
    }
)

In [8]:
index = "test_mapping2"

In [9]:
ed.pandas_to_eland(
    df,
    es_client,
    es_dest_index=index,
    es_if_exists="append",
    es_refresh=True,
    use_pandas_index_for_es_ids=False
)

/home/sidhu/.local/share/virtualenvs/eland-JIur908e/lib/python3.8/site-packages/elasticsearch/connection/base.py:200: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.16/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)
/home/sidhu/.local/share/virtualenvs/eland-JIur908e/lib/python3.8/site-packages/elasticsearch/connection/base.py:200: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.16/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


,file.hash.sha1,file.hash.sha256,label,process.name
GLBDBX4BpoMDJ9chv_8q,hash2,hash1,1,process1


In [11]:
final_df = ed.pandas_to_eland(
    df,
    es_client,
    es_dest_index=index,
    es_if_exists="append",
    es_refresh=True,
    use_pandas_index_for_es_ids=False
).to_pandas()

{'process.name': {'type': 'keyword'}, 'file.hash.sha256': {'type': 'keyword'}, 'file.hash.sha1': {'type': 'keyword'}, 'label': {'type': 'long'}} {'file.hash.sha1': {'type': 'keyword'}, 'file.hash.sha256': {'type': 'keyword'}, 'label': {'type': 'long'}, 'process.name': {'type': 'keyword'}}


/home/sidhu/.local/share/virtualenvs/eland-JIur908e/lib/python3.8/site-packages/elasticsearch/connection/base.py:200: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.16/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


In [30]:
data = {
    "file.hash.sha1": ["hash1"],
    "file.hash.sha256": ["hash2"],
    "label": [1],
    "process.name": ["process1"],
}
df1 = pd.DataFrame(data, data)
df1

,file.hash.sha1,file.hash.sha256,label,process.name
file.hash.sha1,hash1,hash2,1,process1
file.hash.sha256,hash1,hash2,1,process1
label,hash1,hash2,1,process1
process.name,hash1,hash2,1,process1


In [22]:
final_df.iloc[[1]]

,file.hash.sha1,file.hash.sha256,label,process.name
GbBDBX4BpoMDJ9ch2P-M,hash2,hash1,1,process1


In [69]:
dest_mapping = es_client.indices.get_mapping(index=index)[
                index
            ]

In [70]:
dest_mapping

{'mappings': {'properties': {'file': {'properties': {'hash': {'properties': {'sha1': {'type': 'keyword'},
       'sha256': {'type': 'keyword'}}}}},
   'label': {'type': 'long'},
   'process': {'properties': {'name': {'type': 'keyword'}}}}}}

In [101]:
def extract_column_names(d, col_name):

    for k,v in d.items():
        if k == "type" and not isinstance(v, dict):
            return col_name
         
        if k != "properties":
            col_name = f"{col_name}.{k}"

    col_name = extract_column_names(v, col_name)

    return col_name

In [111]:
df = pd.json_normalize(dest_mapping["mappings"]["properties"])

In [128]:
records = {"mappings": {"properties": {}}}

for col,col_type in zip(df.columns, df.iloc[0]):
    col_name = col.replace(".properties", "").replace(".type","")
    records["mappings"]["properties"][col_name] = {"type": col_type}



In [130]:
records

{'mappings': {'properties': {'file.hash.sha1': {'type': 'keyword'},
   'file.hash.sha256': {'type': 'keyword'},
   'label': {'type': 'long'},
   'process.name': {'type': 'keyword'}}}}

In [100]:
dest_mapping["mappings"]["properties"]["file"]["properties"]["hash"]

{'properties': {'sha1': {'type': 'keyword'}, 'sha256': {'type': 'keyword'}}}

In [51]:
".".join(["label"])

'label'